# Stage of Mitosis Detection in Cell

Mitosis is when a mother cell splits into two genetically similar daughter cells. There are several stages to this:
- Interphase
- Prophase
- Metaphase
- Anaphase
- Telophase
- Cytokensis

The main objective of this project is to be able to identify when cell is in:
- Interphase
- Metaphase
- Anaphase

# Methods:

### compareROI: Compare the nucleus in the video to our reference image to determine mitosis stage
#### Parameters:
- Binary mask of frame from video
- x coordinate of bounding rectangle of nucleus
- y coordinate of bounding rectangle of nucleus
- width of bounding rectangle of nucleus
- height of bounding rectangle of nucleus

#### Algorithm:
- Crop ROI from mask of video
- For each reference image compare the cropped ROI to the shape of the reference image using cv2.matchShapes
- If it's the first image then set that as the closest match and save the index
- If matchShapes returns smaller value then change the value
- Return the index of the closest match

This function is called for each frame of the video

# The Algorithm

1. Process reference images
2. Load the video
3. Identify cells/nucleus using thresholding and contours
4. Box the cells
5. Compare boxed cells to reference image using matchShapes in compareROI
6. Name the stage of mitosis of cell based on the index returned from compareROI
7. Repeat for each frame until finished

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
import easygui

ref_strings = ['interphase_ref2_focus.jpg', 'metaphase_ref_focus.jpg', 'anaphase_ref_focus.jpg']
ref_images = []

for i in range(0,len(ref_strings)):
    ref_image = cv2.imread(ref_strings[i])
    G = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(G, 127, 255, cv2.THRESH_BINARY_INV)
    shape = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    CloseRef = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, shape, iterations=2)
    OpenRef = cv2.morphologyEx(CloseRef, cv2.MORPH_OPEN, shape, iterations=2)
    OpenRef = cv2.resize(OpenRef, (100,100))
    ref_images.append(OpenRef)

def compareROI(mask, roix, roiy, roiw, roih): # Compares binary images for shape, smaller diff value means more of a match
    match = 0
    ROI = cv2.resize(mask[roiy:roiy+roih, roix:roix+roiw], (100,100))
    for i in range(0,len(ref_images)):
        diff = cv2.matchShapes(ROI, ref_images[i], 1, 0.0)
        
        if i == 0 or match > diff:
            match = diff
            index = i
            
    return index

# Video capture from a file:
vid = easygui.fileopenbox(filetypes=[".mp4"])
Video = cv2.VideoCapture(vid)
(check, I) = Video.read()

stage_string = ['Interphase', 'Metaphase', 'Anaphase']

while check:
    ret,thresh = cv2.threshold(I[:,:,2], 100, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) # Otsu is auto-global thresholding
    shape = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilated = cv2.dilate(thresh, shape, iterations=2)
    Close = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, shape)
    contours = cv2.findContours(Close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    #I = cv2.drawContours(I, cnts, -1, (0, 255, 0), 1)
    for c in contours: # for each contour bound a rectangle and send for comparison
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(I, (x, y), (x + w, y + h), (255,0,0), 2)
        stage = compareROI(Close, x, y, w, h)
        cv2.putText(I, stage_string[stage], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
    #cut = cv2.bitwise_and(I, I, mask=Close)
    
    cv2.imshow("mask", Close)
    cv2.imshow("frame", I)
    #cv2.imshow("og", I)
    
    # This delays while waiting for a key to be pressed
    key = cv2.waitKey(100)

    # if the 'q' key is pressed, quit:
    if key == ord("q"):
         break

    # Next Frame:
    (check, I) = Video.read()

Video.release()
cv2.waitKey(0)
cv2.destroyAllWindows()